In [2]:
# Step 1: Install pgmpy
!pip install -q pgmpy==0.1.26

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

Define the Bayesian Network structure

In [4]:
model = BayesianNetwork([("guest", "monty"), ("prize", "monty")])

Define CPDs

In [5]:
cpd_guest = TabularCPD(
    variable="guest",
    variable_card=3,
    values=[[1/3], [1/3], [1/3]],
    state_names={"guest": ["A", "B", "C"]}
)

cpd_prize = TabularCPD(
    variable="prize",
    variable_card=3,
    values=[[1/3], [1/3], [1/3]],
    state_names={"prize": ["A", "B", "C"]}
)

Conditional probability for Monty's choice given guest and prize

In [6]:
cpd_monty = TabularCPD(
    variable="monty",
    variable_card=3,
    values=[
        # Monty = A, B, C (rows)
        # guest, prize combinations are in order (A,A), (A,B), (A,C), (B,A), (B,B), (B,C), (C,A), (C,B), (C,C)
        [0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, 1.0, 0.5],  # Monty = A
        [0.5, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.5],  # Monty = B
        [0.5, 1.0, 0.0, 1.0, 0.5, 0.0, 0.0, 0.0, 0.0],  # Monty = C
    ],
    evidence=["guest", "prize"],
    evidence_card=[3, 3],
    state_names={"monty": ["A", "B", "C"], "guest": ["A", "B", "C"], "prize": ["A", "B", "C"]}
)

Add CPDs to model

In [7]:
model.add_cpds(cpd_guest, cpd_prize, cpd_monty)

Verify model consistency

In [8]:
assert model.check_model()

Inference Engine

In [9]:
inference = VariableElimination(model)

In [10]:
print("🔹 Run 1: Probability of specific states")
q1 = inference.query(variables=["monty"], evidence={"guest": "A", "prize": "B"})
print(q1, "\n")

🔹 Run 1: Probability of specific states
+----------+--------------+
| monty    |   phi(monty) |
+==========+==============+
| monty(A) |       0.0000 |
+----------+--------------+
| monty(B) |       0.0000 |
+----------+--------------+
| monty(C) |       1.0000 |
+----------+--------------+ 



In [11]:
print("🔹 Run 2: Probability of initially choosing door A")
q2 = inference.query(variables=["guest"])
print(q2, "\n")

🔹 Run 2: Probability of initially choosing door A
+----------+--------------+
| guest    |   phi(guest) |
+==========+==============+
| guest(A) |       0.3333 |
+----------+--------------+
| guest(B) |       0.3333 |
+----------+--------------+
| guest(C) |       0.3333 |
+----------+--------------+ 



In [12]:
print("🔹 Run 3: When guest chooses A and Monty opens C, what's the probability that the prize is behind each door?")
q3 = inference.query(variables=["prize"], evidence={"guest": "A", "monty": "C"})
print(q3, "\n")

🔹 Run 3: When guest chooses A and Monty opens C, what's the probability that the prize is behind each door?
+----------+--------------+
| prize    |   phi(prize) |
+==========+==============+
| prize(A) |       0.3333 |
+----------+--------------+
| prize(B) |       0.6667 |
+----------+--------------+
| prize(C) |       0.0000 |
+----------+--------------+ 



In [13]:
print("🔹 Run 4: When guest chooses A and Monty opens B, what's the probability of prize?")
q4 = inference.query(variables=["prize"], evidence={"guest": "A", "monty": "B"})
print(q4, "\n")

🔹 Run 4: When guest chooses A and Monty opens B, what's the probability of prize?
+----------+--------------+
| prize    |   phi(prize) |
+==========+==============+
| prize(A) |       0.3333 |
+----------+--------------+
| prize(B) |       0.0000 |
+----------+--------------+
| prize(C) |       0.6667 |
+----------+--------------+ 

